In [110]:
#definir les packages
import pandas as pd
import requests
from tqdm import tqdm

In [111]:
#dans un premier temps, on definit l'url de l'api
limit = 20
#on formate notre url pour pouvoir changer nos valeurs si on le souhaite par la suite , ex : changer l'année ou le pays
url = f"https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/repertoire-national-des-associations-ile-de-france/records?limit={limit}&refine=position%3A%22Active%22&q=sport"
print(url)

#requeter l'api
response = requests.get(url)
print(response.text)

#recuperer le contenu
data = response.json()
df = pd.DataFrame(data)

#transformer json en dataframe
df = pd.json_normalize(data, record_path= 'results')
print(df.head(20).to_markdown())
print("====================")

https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/repertoire-national-des-associations-ile-de-france/records?limit=20&refine=position%3A%22Active%22&q=sport
{"total_count": 301390, "results": [{"id": "W943009831", "id_ex": "0943004817", "siret": null, "rup_mi": null, "management": "943S", "creation_date": "2000-07-04", "declaration_date": "2000-07-04", "publication_date": "2000-07-29", "dissolution_date": null, "nature": "D", "group": "Simple", "title": "ASSOCIATION DES MEDIAS CHINOIS D'EUROPE", "short_title": "ASSOCIATION DES MEDIAS CHINOIS D'E...", "object": "faire rayonner la culture chinoise, promouvoir la cooperation et les echanges culturels sino-europeens encourager l integration et mieux servir les chinois d outre mer tout en protegeant leurs interets.", "social_object1": "006000", "social_object2": "005005", "comp_address_asso": null, "street_number_asso": "110 bis", "street_type_asso": "avenue", "street_name_asso": "de Paris", "namedplace_address_asso": null, "com

In [ ]:
#recuperer tous le contenu de la restriction limit :
limit_df = []
for depart in tqdm(range(0,301390, 20)):
    url = f"https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/repertoire-national-des-associations-ile-de-france/records?limit={limit}&refine=position%3A%22Active%22&q=sport"

    response = requests.get(url)
    data = response.json()
    df = pd.json_normalize(data, record_path="results")
    limit_df.append(df)

df = pd.concat(limit_df)
#display(df)
print(df.shape)

#etre sur d'avoir les bons index 
df.reset_index(inplace=True, drop=True)

#colonnes a garder :  title |  object  |  street_number_asso   | street_type_asso   | street_name_asso   | namedplace_address_asso | pc_address_asso | com_name_asso  | website | observation  | update_date  | dep_name  | reg_name  |   dep_code  |   geo_point_2d.lon |   geo_point_2d.lat |
df = df[['title', 'object', 'street_number_asso', 'street_type_asso', 'street_name_asso', 'pc_address_asso', 'com_name_asso', 'website', 'dep_name','dep_code', 'geo_point_2d.lon', 'geo_point_2d.lat']]
df.head()

#convertir la colonne numero rue en str
df['street_number_asso']= df['street_number_asso'].astype(str)
df['pc_address_asso'] = df['pc_address_asso'].astype(str)

#créer colonne adresse_complete avec les colonnes street_number_asso   | street_type_asso   | street_name_asso /pc_address_asso | com_name_asso
df['adresse_complete'] = df['street_number_asso'] + ' ' + df['street_type_asso'] + ' ' + df['street_name_asso'] + ' ' + df['pc_address_asso'] + ' ' + df['com_name_asso']

#supprimer les colonnes : street_number_asso   | street_type_asso   | street_name_asso | com_name_asso
df = df.drop(columns=['street_number_asso', 'street_type_asso', 'street_name_asso', 'com_name_asso'])

#renommer colonne : title = nom_asso/club , geo_point_2d.lon = lon , geo_point_2d.lat = lat
df = df.rename(columns={'title': 'nom_asso',
                        'geo_point_2d.lon': 'lon',
                        'geo_point_2d.lat': 'lat',
                        'dep_name': 'nom_ville',
                        'dep_code': 'code_postal'}
                    )

df['website'].fillna(value ='Pas encore de site web actuellement', inplace=True)
df.head()

In [121]:
#ouvrir csv
df = pd.read_csv("repertoire-national-des-associations-ile-de-france.csv", sep=';')
df.head()

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_11632\43719209.py:2: DtypeWarning: Columns (8,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("repertoire-national-des-associations-ile-de-france.csv", sep=';')


,Id,Ancien Id,SIRET,Numéro de RUP,Gestion,Date de création,Date de déclaration,Date de publication,Date de dissolution,Nature,...,Position,Date de Mise à jour,geo_point_2d,Nom Officiel Département,Nom Officiel EPCI,Nom Officiel Région,Code Iso 3166-3 Zone,Code Officiel Département,Code Officiel EPCI,Code Officiel Région
0,W943009831,0943004817,NaN,NaN,943S,2000-07-04,2000-07-04,2000-07-29,NaN,D,...,Active,2021-05-26T11:04:38+02:00,"48.799038, 2.365538",Val-de-Marne,Métropole du Grand Paris,Île-de-France,FXX,94,200054781,11
1,W943009882,0943005827,NaN,NaN,943S,2004-03-29,2004-03-29,2004-04-17,NaN,D,...,Active,2021-06-02T17:14:20+02:00,"48.802671, 2.36228",Val-de-Marne,Métropole du Grand Paris,Île-de-France,FXX,94,200054781,11
2,W943009904,0943005019,NaN,NaN,943S,2001-04-11,2001-04-11,2001-05-05,NaN,D,...,Active,2021-06-04T11:03:40+02:00,"48.766823, 2.382495",Val-de-Marne,Métropole du Grand Paris,Île-de-France,FXX,94,200054781,11
3,W943009911,0943004431,NaN,NaN,943S,1998-12-16,1998-12-16,1999-01-09,NaN,D,...,Active,2021-06-04T13:34:21+02:00,"48.770504, 2.401006",Val-de-Marne,Métropole du Grand Paris,Île-de-France,FXX,94,200054781,11
4,W943009940,NaN,NaN,NaN,943S,2021-06-09,2021-06-09,2021-06-15,NaN,D,...,Active,2021-06-15T16:15:17+02:00,"48.814313, 2.36321",Val-de-Marne,Métropole du Grand Paris,Île-de-France,FXX,94,200054781,11


In [122]:
df.shape

(301390, 45)

In [123]:
df.columns

Index(['Id', 'Ancien Id', 'SIRET', 'Numéro de RUP', 'Gestion',
       'Date de création', 'Date de déclaration', 'Date de publication',
       'Date de dissolution', 'Nature', 'Groupement', 'Titre', 'Titre court',
       'Objet', 'Objet Social 1', 'Objet Social 2',
       'Complément d'adresse Siège', 'Numéro de Voie Siège',
       'Type de Voie Siège', 'Libellé Voie Siège', 'Lieu Dit Siège',
       'Code Officiel Commune Siège', 'Code Postal Siège', 'Nom Commune Siège',
       'Complément Adresse Gestion', 'Complément Géo Adresse Gestion',
       'Libellé Voie Adresse Gestion', 'Lieu Dit Adresse Gestion',
       'Code Postal Adresse Gestion', 'Autre Adresse Gestion',
       'Pays Adresse Gestion', 'Code de la civilité du dirigeant principal',
       'SIte Web', 'Publication sur le web', 'Observation', 'Position',
       'Date de Mise à jour', 'geo_point_2d', 'Nom Officiel Département',
       'Nom Officiel EPCI', 'Nom Officiel Région', 'Code Iso 3166-3 Zone',
       'Code Officiel Dép

In [124]:
df.iloc[0]

Id                                                                                   W943009831
Ancien Id                                                                            0943004817
SIRET                                                                                       NaN
Numéro de RUP                                                                               NaN
Gestion                                                                                    943S
Date de création                                                                     2000-07-04
Date de déclaration                                                                  2000-07-04
Date de publication                                                                  2000-07-29
Date de dissolution                                                                         NaN
Nature                                                                                        D
Groupement                              

In [125]:
df.columns

Index(['Id', 'Ancien Id', 'SIRET', 'Numéro de RUP', 'Gestion',
       'Date de création', 'Date de déclaration', 'Date de publication',
       'Date de dissolution', 'Nature', 'Groupement', 'Titre', 'Titre court',
       'Objet', 'Objet Social 1', 'Objet Social 2',
       'Complément d'adresse Siège', 'Numéro de Voie Siège',
       'Type de Voie Siège', 'Libellé Voie Siège', 'Lieu Dit Siège',
       'Code Officiel Commune Siège', 'Code Postal Siège', 'Nom Commune Siège',
       'Complément Adresse Gestion', 'Complément Géo Adresse Gestion',
       'Libellé Voie Adresse Gestion', 'Lieu Dit Adresse Gestion',
       'Code Postal Adresse Gestion', 'Autre Adresse Gestion',
       'Pays Adresse Gestion', 'Code de la civilité du dirigeant principal',
       'SIte Web', 'Publication sur le web', 'Observation', 'Position',
       'Date de Mise à jour', 'geo_point_2d', 'Nom Officiel Département',
       'Nom Officiel EPCI', 'Nom Officiel Région', 'Code Iso 3166-3 Zone',
       'Code Officiel Dép

In [143]:
df = df[['Titre', 'Objet', 'Numéro de Voie Siège', 'Type de Voie Siège', 'Libellé Voie Siège','Code Officiel Commune Siège','Code Postal Siège', 'Nom Commune Siège', 'SIte Web', 'Nom Officiel Département','Code Officiel Département', 'geo_point_2d']]
df1 = df.copy()

In [136]:
df1.head()

,Titre,Objet,Numéro de Voie Siège,Type de Voie Siège,Libellé Voie Siège,Code Officiel Commune Siège,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d
0,ASSOCIATION DES MEDIAS CHINOIS D'EUROPE,"faire rayonner la culture chinoise, promouvoir...",110 bis,avenue,de Paris,94076,94800.0,Villejuif,NaN,Val-de-Marne,94,"48.799038, 2.365538"
1,FAN-CLUB DE CLAUDE MONET,favoriser et développer promouvoir les activit...,12,rue,Sacco et Vanzetti,94076,94800.0,Villejuif,NaN,Val-de-Marne,94,"48.802671, 2.36228"
2,DEFI PASSION,participation course automobile 24heures Magny...,34,rue,des 15 Arpents,94073,94320.0,Thiais,NaN,Val-de-Marne,94,"48.766823, 2.382495"
3,ASSOCIATION CONFRATERNELLE D'ENTRAIDE MEDICALE,apporter une aide financiere sous forme d'inde...,3,rue,Georgeon,94073,94320.0,Thiais,NaN,Val-de-Marne,94,"48.770504, 2.401006"
4,ASSOCIATION BALIMAYA NI TERIYA KITA ASSOCIATI...,"promotion de la solidarité, l'entraide, la coh...",6,rue,Itzhak Rabin,94043,94270.0,Le Kremlin-Bicêtre,NaN,Val-de-Marne,94,"48.814313, 2.36321"


In [144]:
df1.isnull().sum()

#supprimer les lignes ayany nom commune sieges et code postal siege avec des valeurs manquantes 
df1 = df1.dropna(subset=(['Code Postal Siège','Nom Commune Siège']))

In [116]:
df1.shape

(301323, 9)

In [145]:
#transformer code post siege en int puis str
df1['Code Postal Siège'] = df1['Code Postal Siège'].astype(int)
df1['Code Postal Siège'] = df1['Code Postal Siège'].astype(str)

In [119]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 301323 entries, 0 to 301389
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   Titre                        301322 non-null  object
 1   Objet                        301318 non-null  object
 2   Code Officiel Commune Siège  301323 non-null  object
 3   Code Postal Siège            301323 non-null  object
 4   Nom Commune Siège            301323 non-null  object
 5   SIte Web                     18449 non-null   object
 6   Nom Officiel Département     301323 non-null  object
 7   Code Officiel Département    301323 non-null  int64 
 8   geo_point_2d                 297653 non-null  object
 9   Numéro de Voie Siège         301323 non-null  object
dtypes: int64(1), object(9)
memory usage: 25.3+ MB


In [102]:
df1.iloc[0]

Titre                                    ASSOCIATION DES MEDIAS CHINOIS D'EUROPE
Objet                          faire rayonner la culture chinoise, promouvoir...
Numéro de Voie Siège                                                     110 bis
Type de Voie Siège                                                        avenue
Libellé Voie Siège                                                      de Paris
Code Officiel Commune Siège                                                94076
Code Postal Siège                                                          94800
Nom Commune Siège                                                      Villejuif
SIte Web                                                                     NaN
Nom Officiel Département                                            Val-de-Marne
Code Officiel Département                                                     94
geo_point_2d                                                 48.799038, 2.365538
Name: 0, dtype: object

In [146]:
#convertir la colonne numero rue en str
df1['Numéro de Voie Siège']= df['Numéro de Voie Siège'].astype(str)
df1['Code Officiel Commune Siège'] = df1['Code Officiel Commune Siège'].astype(str)


df1['Adresse'] = df1['Numéro de Voie Siège'] + ' ' + df1['Type de Voie Siège'] + ' ' + df1['Libellé Voie Siège'] + ' ' + df1['Code Postal Siège'] + ' ' + df1['Nom Commune Siège']

df1['SIte Web'].fillna(value ='Pas encore de site web actuellement', inplace=True)
df1.head()

C:\Users\Administrateur\AppData\Local\Temp\ipykernel_11632\1960844287.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['SIte Web'].fillna(value ='Pas encore de site web actuellement', inplace=True)


,Titre,Objet,Numéro de Voie Siège,Type de Voie Siège,Libellé Voie Siège,Code Officiel Commune Siège,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d,Adresse
0,ASSOCIATION DES MEDIAS CHINOIS D'EUROPE,"faire rayonner la culture chinoise, promouvoir...",110 bis,avenue,de Paris,94076,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.799038, 2.365538",110 bis avenue de Paris 94800 Villejuif
1,FAN-CLUB DE CLAUDE MONET,favoriser et développer promouvoir les activit...,12,rue,Sacco et Vanzetti,94076,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.802671, 2.36228",12 rue Sacco et Vanzetti 94800 Villejuif
2,DEFI PASSION,participation course automobile 24heures Magny...,34,rue,des 15 Arpents,94073,94320,Thiais,Pas encore de site web actuellement,Val-de-Marne,94,"48.766823, 2.382495",34 rue des 15 Arpents 94320 Thiais
3,ASSOCIATION CONFRATERNELLE D'ENTRAIDE MEDICALE,apporter une aide financiere sous forme d'inde...,3,rue,Georgeon,94073,94320,Thiais,Pas encore de site web actuellement,Val-de-Marne,94,"48.770504, 2.401006",3 rue Georgeon 94320 Thiais
4,ASSOCIATION BALIMAYA NI TERIYA KITA ASSOCIATI...,"promotion de la solidarité, l'entraide, la coh...",6,rue,Itzhak Rabin,94043,94270,Le Kremlin-Bicêtre,Pas encore de site web actuellement,Val-de-Marne,94,"48.814313, 2.36321",6 rue Itzhak Rabin 94270 Le Kremlin-Bicêtre


In [147]:
df1 = df1.drop(columns=['Numéro de Voie Siège', 'Type de Voie Siège', 'Libellé Voie Siège','Code Officiel Commune Siège'], axis = 1)

In [108]:
df1.shape

(301390, 9)

In [148]:
df1.head(2)

,Titre,Objet,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d,Adresse
0,ASSOCIATION DES MEDIAS CHINOIS D'EUROPE,"faire rayonner la culture chinoise, promouvoir...",94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.799038, 2.365538",110 bis avenue de Paris 94800 Villejuif
1,FAN-CLUB DE CLAUDE MONET,favoriser et développer promouvoir les activit...,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.802671, 2.36228",12 rue Sacco et Vanzetti 94800 Villejuif


In [151]:
df1['Objet'].info()

<class 'pandas.core.series.Series'>
Index: 301323 entries, 0 to 301389
Series name: Objet
Non-Null Count   Dtype 
--------------   ----- 
301318 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


In [153]:
#recuperer seulement les lignes qui contiennent les caracteres : sport ou club ou pratique ou ball :
df_sport = df1[df1[['Objet', 'Titre']].apply(lambda x: x.str.contains('sport|club|pratique|ball', case=False, na=False)).any(axis=1)]
df_sport.shape

df_sport.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77390 entries, 1 to 301387
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Titre                      77390 non-null  object
 1   Objet                      77390 non-null  object
 2   Code Postal Siège          77390 non-null  object
 3   Nom Commune Siège          77390 non-null  object
 4   SIte Web                   77390 non-null  object
 5   Nom Officiel Département   77390 non-null  object
 6   Code Officiel Département  77390 non-null  int64 
 7   geo_point_2d               76417 non-null  object
 8   Adresse                    71790 non-null  object
dtypes: int64(1), object(8)
memory usage: 5.9+ MB


In [154]:
df_sport.head(100)

,Titre,Objet,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d,Adresse
1,FAN-CLUB DE CLAUDE MONET,favoriser et développer promouvoir les activit...,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.802671, 2.36228",12 rue Sacco et Vanzetti 94800 Villejuif
5,ZEBULON CYCLE VERT,contribuer au développement sportif et pédagog...,94550,Chevilly-Larue,Pas encore de site web actuellement,Val-de-Marne,94,"48.771525, 2.366082",23 avenue Georges Brassens 94550 Chevilly-Larue
6,ALLIANCE FRAICHEUR,promotion et le developpement des fruits et le...,94150,Rungis,Pas encore de site web actuellement,Val-de-Marne,94,NaN,3 rue de la Corderie CENTRA 356 94150 Rungis
8,LES AMIS DE L'ECOLOGIE DANS LE MONDE ARABE,promouvoir l écologie dans le monde arabe sout...,94270,Le Kremlin-Bicêtre,Pas encore de site web actuellement,Val-de-Marne,94,"48.806213, 2.3504",53 avenue Charles Gide 94270 Le Kremlin-Bic...
9,'LE TOLSTOI',création d un club de fléchettes,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.798207, 2.35461",2 rue Tolstoi 94800 Villejuif
...,...,...,...,...,...,...,...,...,...
294,CBB RACING,"défendre, développer, promouvoir l'apprentissa...",77340,Pontault-Combault,Pas encore de site web actuellement,Seine-et-Marne,77,"48.810394, 2.596345",135 rue de Nogent 77340 Pontault-Combault
296,LES XVIÈMES PONTELLOIS,"promouvoir la pratique et l'image du Rugby, de...",77340,Pontault-Combault,Pas encore de site web actuellement,Seine-et-Marne,77,"48.785433, 2.613305",7 rue Victor Hugo 77340 Pontault-Combault
300,CRÉCYCURIENS,l'objet de l'association : -Promouvoir les bon...,77580,Crécy-la-Chapelle,Pas encore de site web actuellement,Seine-et-Marne,77,"48.841275, 2.913481",43 rue du Champ de l Eau 77580 Crécy-la-Chapelle
304,FORMULE JUSTICE BINGA,"aides et assistances aux populations démunies,...",77500,Chelles,Pas encore de site web actuellement,Seine-et-Marne,77,"48.8738, 2.59446",5 rue Alice et Jean Lafont 77500 Chelles


In [156]:
#sup les lignes contenant "fan-club" dans la colonne "Objet" ou "titre"
df_sport = df_sport[~df_sport[['Objet', 'Titre']].apply(lambda x: x.str.contains('fan-club', case=False, na=False)).any(axis=1)]

In [157]:
df_sport.head(10)

,Titre,Objet,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d,Adresse
5,ZEBULON CYCLE VERT,contribuer au développement sportif et pédagog...,94550,Chevilly-Larue,Pas encore de site web actuellement,Val-de-Marne,94,"48.771525, 2.366082",23 avenue Georges Brassens 94550 Chevilly-Larue
6,ALLIANCE FRAICHEUR,promotion et le developpement des fruits et le...,94150,Rungis,Pas encore de site web actuellement,Val-de-Marne,94,NaN,3 rue de la Corderie CENTRA 356 94150 Rungis
8,LES AMIS DE L'ECOLOGIE DANS LE MONDE ARABE,promouvoir l écologie dans le monde arabe sout...,94270,Le Kremlin-Bicêtre,Pas encore de site web actuellement,Val-de-Marne,94,"48.806213, 2.3504",53 avenue Charles Gide 94270 Le Kremlin-Bic...
9,'LE TOLSTOI',création d un club de fléchettes,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.798207, 2.35461",2 rue Tolstoi 94800 Villejuif
10,ACL RACING KART,pratique du karting de compétition,94240,L' Haÿ-les-Roses,Pas encore de site web actuellement,Val-de-Marne,94,"48.782464, 2.33589",20 voie des Saussaies 94240 L' Haÿ-les-Roses
14,ASSOCIATION DES AMIS DE LA REGION DE VINHAIS E...,promotion de la région en ce qui concerne ses ...,94320,Thiais,Pas encore de site web actuellement,Val-de-Marne,94,"48.76045, 2.389266",40 sentier du Paradis 94320 Thiais
16,ARCUEIL FOOTBALL CLUB,promouvoir la pratique du football,94110,Arcueil,Pas encore de site web actuellement,Val-de-Marne,94,"48.812653, 2.331506",69 avenue Lénine 94110 Arcueil
21,DANSE BIEN-ÊTRE SPORT--SANTÉ DB-É-S-S,proposer une activité physique dans le cadre d...,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.802076, 2.357377",18 allée du Vercors 94800 Villejuif
22,AQUASPORT BIEN-ÊTRE,organiser par tous les moyens et de proposer à...,94230,Cachan,Pas encore de site web actuellement,Val-de-Marne,94,"48.794483, 2.336186",17 rue Guichard 94230 Cachan
30,CLUB DES AMIS,entraide et développement de relations familiales,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.78637, 2.360578",7 rue Honoré de Balzac 94800 Villejuif


In [158]:
#faire reset index 
df_sport.reset_index(inplace=True, drop=True)
df_sport.head()

,Titre,Objet,Code Postal Siège,Nom Commune Siège,SIte Web,Nom Officiel Département,Code Officiel Département,geo_point_2d,Adresse
0,ZEBULON CYCLE VERT,contribuer au développement sportif et pédagog...,94550,Chevilly-Larue,Pas encore de site web actuellement,Val-de-Marne,94,"48.771525, 2.366082",23 avenue Georges Brassens 94550 Chevilly-Larue
1,ALLIANCE FRAICHEUR,promotion et le developpement des fruits et le...,94150,Rungis,Pas encore de site web actuellement,Val-de-Marne,94,NaN,3 rue de la Corderie CENTRA 356 94150 Rungis
2,LES AMIS DE L'ECOLOGIE DANS LE MONDE ARABE,promouvoir l écologie dans le monde arabe sout...,94270,Le Kremlin-Bicêtre,Pas encore de site web actuellement,Val-de-Marne,94,"48.806213, 2.3504",53 avenue Charles Gide 94270 Le Kremlin-Bic...
3,'LE TOLSTOI',création d un club de fléchettes,94800,Villejuif,Pas encore de site web actuellement,Val-de-Marne,94,"48.798207, 2.35461",2 rue Tolstoi 94800 Villejuif
4,ACL RACING KART,pratique du karting de compétition,94240,L' Haÿ-les-Roses,Pas encore de site web actuellement,Val-de-Marne,94,"48.782464, 2.33589",20 voie des Saussaies 94240 L' Haÿ-les-Roses


In [159]:
df_sport.to_csv("sports_clubs_ile_de_france.csv", sep=';', index=False)